## Tarea 2 - Machine Learning I
El objetivo es diseñar una red neuronal que sea capaz de predecir el valor de la precipitación mediante la información de variables de larga escala. El dataset consiste en 450 predictores y un predictando, la precipitación en Madrid.
Hay que tener en cuenta que la precipitación es una variable mixta, ya que habrá días con un valor exacto de 0 y otros que se encuentren en el intervalo (0,inf).
Así pues, diseñar una red neuronal con KERAS, que sea capaz de predecir la precipitación de acuerdo con los siguientes índices:
- La ocurrencia de precipitación se evalúa con el AUC. 
- La cantidad de precipitación se evalúa con el MSE.

Podréis valeros de todas las técnicas aprendidas hasta ahora: early stopping, regularización, backpropagation + momento, ajustar el learning rate, poner varias capas y demás. Esto no quiere decir que la red que contenga toda esta variedad de técnicas será la que obtenga un menor error. Tenéis que jugar y probar distintos diseños de redes con el fin de encontrar la que consideréis que obtendría un menor error en un dataset distinto (es decir, que tenga buena capacidad de generalización).

Construiréis vuestro modelo usando xTrain e yTrain, y después evaluaréis el modelo en xTest. La predicción que resulte al evaluar el modelo en xTest es lo que tenéis que guardar en un fichero y lo que se envía para corrección:
save(yTest_ocu, yTest_cantidad,  file = “1erApellido_Nombre.rda”)
e.g., save(yTest_ocu, yTest_cantidad, file = “Baño_Jorge.rda”)


**NOTA: se considera día de lluvia si el valor de precipitación es mayor que 1, sino convertir a 0.**

Además deberéis de responder a las siguiente cuestiones:

**He resuelto el problema en RSTUDIO por problemas con keras en jupyer. 
Dejo aqui el código usado en R**

1- ¿Cómo has preprocesado los datos y por qué (por ejemplo, has estandarizado el predictando o los predictores)? **Descríbe** el proceso en 3-4 líneas.

**1.1. Lectura/Carga de fichero de Datos**

**1.2. Preprocesado de datos. Normalizacion de los datos en base al dataSet Train**

In [ ]:
#Es importante el orden en el cual normalizamos los datos. 
#Primero he normalizado el conjunto de Test con el conjunto de Train. 
#Después he normalizado el conjunto de Train con el mismo.
#He seguido este proceso para no alterar el conjunto Test

2- ¿Qué topología de red neuronal has encontrado óptima para la **ocurrencia de precipitación**? **Describe** en 3-4 líneas por qué has usado esta topología e incluye el **código de Keras** (mirar ejemplo)

In [ ]:
### Describe en 3-4 líneas ----------------------

#He probado diferentes tipos de arquitecturas pero al final me he decantado por una arquitectura de tipo VAE
#para ambos casos.

#Primero tenemos una capa de 10 unidades
#En la siguiente capa la red se divide en dos.
#Cada una de estas partes esta compuesta en 20 unidades
#En la siguiente capa se unen las dos partes anteriores
#Se usa una función de activación sigmoide para clasificación

In [1]:
model_class <- keras_model_sequential() 

#Topologia tipo VAE
inputs = layer_input(shape = dimensiones_train[2])
#Ponemos el numero de columnas de mnist
x = inputs
l1 = layer_dense(x,units = 10, activation = "sigmoid")
l2a = layer_dense(l1,units = 20, activation = "sigmoid")
l2b = layer_dense(l1,units = 20)
l3 = layer_add(list(l2a,l2b))#combinacion lineal
l3_activated = layer_activation(l3,activation = "sigmoid")
outputs = layer_dense(l3_activated,units = 1, activation = "sigmoid")


ERROR: Error in keras_model_sequential(): could not find function "keras_model_sequential"


In [ ]:
# model <- keras_model(inputs = inputs, outputs = outputs)
model_class <- keras_model(inputs = inputs, outputs = outputs)

model_class %>% compile(loss = 'binary_crossentropy',
                  optimizer = optimizer_sgd(learning_rate = 0.01), 
                  metrics = 'accuracy')

callbacks = list(callback_early_stopping(patience = 15),
                 callback_model_checkpoint(filepath=paste0('model1_occ.h5'),
                 monitor='accuracy', save_best_only=TRUE))

history <- model_class %>% fit(xTrain, yTrain_occ, epochs = 10000, 
                         batch_size = 128, 
                         verbose = 1, callbacks=callbacks)   

plot(history)

2- ¿Qué topología de red neuronal has encontrado óptima para la **cantidad de precipitación**? **Describe** en 3-4 líneas por qué has usado esta topología e incluye el **código de Keras** (mirar ejemplo)

In [ ]:
#La misma red neuronal que en el caso anterior. 

#Cambiando la funcion loss en el compilar 

#y en la ultima capa se pone una función linear
#para regresión

In [ ]:
model_pred <- keras_model_sequential() 
#Topologia tipo VAE
inputs = layer_input(shape = dimensiones_train[2])
x = inputs
l1 = layer_dense(x,units = 10, activation = "sigmoid")
l2a = layer_dense(l1,units = 20, activation = "sigmoid")
l2b = layer_dense(l1,units = 20)
l3 = layer_add(list(l2a,l2b))#combinacion lineal
l3_activated = layer_activation(l3,activation = "sigmoid")
outputs = layer_dense(l3_activated,units = 1, activation = "linear")


In [ ]:
model_pred <- keras_model(inputs = inputs, outputs = outputs)

model_pred %>% compile(loss = 'mse',
                       optimizer = optimizer_sgd(learning_rate=0.01))

callbacks = list(callback_early_stopping(patience = 15),
                 callback_model_checkpoint(filepath=paste0('model1_reg.h5'),
                                           monitor='loss', save_best_only=TRUE))

history_pred <- model_pred %>% fit(xTrain, yTrain, epochs = 10000, 
                         batch_size = 128, 
                         verbose = 1, callbacks=callbacks)   

plot(history_pred)